## Import Libraries

In [49]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import zeros, newaxis
import cv2

import matplotlib.pyplot as plt
import seaborn
import random
from sklearn.preprocessing import StandardScaler


random.seed(42)

## Read Data

In [50]:
axial_image = 'MRNet-v1.0/train/axial/0000.npy'
coronal_image = 'MRNet-v1.0/train/coronal/0000.npy'
saggital_image = 'MRNet-v1.0/train/sagittal/0000.npy'


img_array_axial = np.load(axial_image)
img_array_coronal = np.load(coronal_image)
img_array_sagittal = np.load(saggital_image)

In [51]:
abnormal_path_train =  'MRNet-v1.0/train-abnormal.csv'
acl_path_train = 'MRNet-v1.0/train-acl.csv'
meniscus_train =  'MRNet-v1.0/train-meniscus.csv'


abnormal_path_valid =  'MRNet-v1.0/valid-abnormal.csv'
acl_path_valid = 'MRNet-v1.0/valid-acl.csv'
meniscus_valid =  'MRNet-v1.0/valid-meniscus.csv'


In [52]:
def load_labels(abnormal_path, acl_path, meniscus_path):
    
    df_labels_abnormal =  pd.read_csv(abnormal_path, names=['Image_Name', 'Abnormal_Label'])
    df_labels_acl = pd.read_csv(acl_path, names=['Image_Name', 'Acl_Label'])
    df_labels_meniscus = pd.read_csv(meniscus_path, names=['Image_Name', 'Meniscus_Label'])
    
    df_lables_combined = pd.merge(df_labels_abnormal, df_labels_acl, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined = pd.merge(df_lables_combined, df_labels_meniscus, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].astype(str)
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].apply(lambda x: x.zfill(4))
    
    return df_lables_combined

In [53]:
# Load Train Labels
df_labels_train = load_labels(abnormal_path_train, acl_path_train, meniscus_train)
df_labels_train

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,0000,1,0,0
1,0001,1,1,1
2,0002,1,0,0
3,0003,1,0,1
4,0004,1,0,0
...,...,...,...,...
1125,1125,1,0,1
1126,1126,1,0,1
1127,1127,0,0,0
1128,1128,1,0,0


In [54]:
# Load Valid Labels
df_labels_valid = load_labels(abnormal_path_valid, acl_path_valid, meniscus_valid)
df_labels_valid

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,1130,0,0,0
1,1131,0,0,0
2,1132,0,0,0
3,1133,0,0,0
4,1134,0,0,0
...,...,...,...,...
115,1245,1,1,1
116,1246,1,1,1
117,1247,1,0,1
118,1248,1,1,1


In [55]:
glob.glob('MRNet-v1.0\*')

['MRNet-v1.0\\train',
 'MRNet-v1.0\\train-abnormal.csv',
 'MRNet-v1.0\\train-acl.csv',
 'MRNet-v1.0\\train-meniscus.csv',
 'MRNet-v1.0\\valid',
 'MRNet-v1.0\\valid-abnormal.csv',
 'MRNet-v1.0\\valid-acl.csv',
 'MRNet-v1.0\\valid-meniscus.csv']

In [56]:
# Read Traning Data
df_labels_train.pop("Image_Name")
def read_data(train_path, df_labels):
    axial_images = []
    coronal_images = []
    sagittal_images = []
    
    # train_path = '/kaggle/input/mrnet-v1/MRNet-v1.0/train'
    train_folders = glob.glob(train_path + '/*')
    
    for folder in train_folders:
        train_data = glob.glob(folder + '/*.npy')
        print('Total samples in data:', len(train_data))
        print('Loading Data From :   ', folder)
        
        for i in train_data:
#             print(i)
            img_array = np.load(i)
            start, end = (img_array.shape[0]/2)-8, (img_array.shape[0]/2)+8
            image_tensor = img_array[int(start):int(end),:,:].reshape(256,256,16)
            if 'axial' in i:
                axial_images.append(image_tensor)
            if 'coronal' in i:
                coronal_images.append(image_tensor)
            if 'sagittal' in i:
                sagittal_images.append(image_tensor)
        print('Loading Complete From :   ', folder)
        print('\n\n')
    return axial_images, coronal_images, sagittal_images

train_path =  'MRNet-v1.0\\train'
valid_path =  'MRNet-v1.0\\valid'

axial_images_train, coronal_images_train, sagittal_images_train = read_data(train_path, df_labels_train)

Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\axial
Loading Complete From :    MRNet-v1.0\train\axial



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\coronal
Loading Complete From :    MRNet-v1.0\train\coronal



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\sagittal
Loading Complete From :    MRNet-v1.0\train\sagittal





In [57]:
yTrain = df_labels_train.pop('Abnormal_Label')
yTest = df_labels_valid.pop('Abnormal_Label')

In [58]:
print(np.array(axial_images_train).shape)
print(np.array(coronal_images_train).shape)
print(np.array(sagittal_images_train).shape)

(1130, 256, 256, 16)
(1130, 256, 256, 16)
(1130, 256, 256, 16)


In [59]:
# #Preprocess
# scale= StandardScaler()
# xTrain = [np.array(coronal_images_train), np.array(sagittal_images_train), np.array(axial_images_train)]
# # standardization of dependent variables
# scaled_data = scale.fit_transform(xTrain) 
# print(scaled_data)

## Modeling

In [60]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K


In [61]:
# Three Input Model

def model ():
    visible1 = Input(shape=(256,256,16))
    
    conv11 = Conv2D( filters=15, kernel_size=(5,5), activation="relu")(visible1)
    pool11 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv11)
    
    conv12 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(pool11)
    pool12 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv12)
    
    # conv13 = Conv2D(filters=16, kernel_size=(5,5), activation='relu')(pool12)
    # pool13 = MaxPooling2D(pool_size=(2, 2),strides=(2,2))(conv13)
    
    flat1 = Flatten()(pool12)
    
    return flat1, visible1
model()

# merge input models
model_1, Visible_1 = model()
model_2, Visible_2 = model()
model_3, Visible_3 = model()

merge = concatenate([model_1, model_2, model_3])

hidden1 = Dense(100, activation='relu')(merge)
hidden2 = Dense(50, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)
model = Model(inputs=[Visible_1, Visible_2, Visible_3], outputs=output)

print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 input_11 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 input_12 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                6)]                                                         

In [62]:
lr = 0.00005
momentum = 0.6
epochs = 10
model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.SGD( learning_rate = lr, momentum = momentum),metrics=["AUC"])

model.fit(x = [np.array(coronal_images_train), np.array(sagittal_images_train), np.array(axial_images_train)],
          y = yTrain,
          epochs=epochs,
          batch_size=32,
          validation_split=0.20)


Epoch 1/10
29/29 [==============================] - 332s 11s/step - loss: 32.1069 - auc: 0.5508 - val_loss: 0.5782 - val_auc: 0.7402
Epoch 2/10
29/29 [==============================] - 353s 12s/step - loss: 0.4346 - auc: 0.8144 - val_loss: 0.5145 - val_auc: 0.7619
Epoch 3/10
29/29 [==============================] - 347s 12s/step - loss: 0.3509 - auc: 0.8667 - val_loss: 0.4524 - val_auc: 0.7856
Epoch 4/10
29/29 [==============================] - 329s 11s/step - loss: 0.2635 - auc: 0.9255 - val_loss: 0.9538 - val_auc: 0.7987
Epoch 5/10
29/29 [==============================] - 351s 12s/step - loss: 0.2129 - auc: 0.9590 - val_loss: 0.5587 - val_auc: 0.7979
Epoch 6/10
29/29 [==============================] - 341s 12s/step - loss: 0.1660 - auc: 0.9812 - val_loss: 0.4595 - val_auc: 0.7884
Epoch 7/10
29/29 [==============================] - 356s 12s/step - loss: 0.1430 - auc: 0.9884 - val_loss: 0.4636 - val_auc: 0.7974
Epoch 8/10
29/29 [==============================] - 325s 11s/step - loss: 0

In [63]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12815122338539173357
xla_global_id: -1
]


In [64]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [65]:
axial_images_valid, coronal_images_valid, sagittal_images_valid = read_data(valid_path, df_labels_valid.astype('float32'))

Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\axial
Loading Complete From :    MRNet-v1.0\valid\axial



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\coronal
Loading Complete From :    MRNet-v1.0\valid\coronal



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\sagittal
Loading Complete From :    MRNet-v1.0\valid\sagittal





In [66]:
# # preprocess testing data
# xTest = [ np.array(coronal_images_valid), np.array(sagittal_images_valid), np.array(axial_images_valid)]
# # standardization of dependent variables
# scaled_data = scale.fit_transform(xTest) 
# print(scaled_data)

## Evaluate Model

In [67]:
y_pred = model.predict([ np.array(coronal_images_valid), np.array(sagittal_images_valid), np.array(axial_images_valid)])

4/4 [==============================] - 11s 3s/step


In [68]:
y_pred

array([[0.37541425],
       [0.43885952],
       [0.6879462 ],
       [0.41511682],
       [0.99972355],
       [0.99856246],
       [0.9980773 ],
       [0.9779097 ],
       [0.4921853 ],
       [0.6797662 ],
       [0.9999096 ],
       [0.98820555],
       [0.99337304],
       [0.904919  ],
       [0.9990809 ],
       [0.92524564],
       [0.879166  ],
       [0.39913   ],
       [0.98601294],
       [0.586687  ],
       [0.83671045],
       [0.99999875],
       [0.03525754],
       [0.22274464],
       [0.99106485],
       [0.99320304],
       [0.9999908 ],
       [0.91709584],
       [0.99979097],
       [0.9746872 ],
       [0.96608007],
       [0.9553835 ],
       [0.82252854],
       [0.9803484 ],
       [0.03257326],
       [0.9505714 ],
       [0.9941634 ],
       [0.99655753],
       [0.96646565],
       [0.9791042 ],
       [0.8356404 ],
       [0.9993565 ],
       [0.9386762 ],
       [0.99705225],
       [0.9938251 ],
       [0.83953243],
       [0.9703093 ],
       [0.995

In [69]:
from sklearn.metrics import classification_report
# 
# print(classification_report(y_pred.round(), axial_labels_valid))

In [70]:
from sklearn.metrics import roc_auc_score, accuracy_score
ypreds_abnormal_soft = list(y_pred[:,0])
ypreds_abnormal_hard = list(np.array(y_pred[:,0]).round())
ytrue_abnormal = yTest #(np.array(df_labels_valid)[:,0])
auc_abnormal = roc_auc_score(y_score= ypreds_abnormal_soft, y_true= ytrue_abnormal)
accuracy_abnormal = accuracy_score(y_true= ytrue_abnormal, y_pred= ypreds_abnormal_hard)

# ypreds_acl_soft = list(y_pred[:,1])
# ypreds_acl_hard = list(np.array(y_pred[:,1]).round())
# ytrue_acl = list(np.array(df_labels_valid)[:,1])
# auc_acl = roc_auc_score(y_score= ypreds_acl_soft, y_true= ytrue_acl)
# accuracy_acl = accuracy_score(y_true= ytrue_acl, y_pred= ypreds_acl_hard)


# ypreds_meniscus_soft = list(y_pred[:,2])
# ypreds_meniscus_hard = list(np.array(y_pred[:,2]).round())
# ytrue_meniscus = list(np.array(df_labels_valid)[:,2])
# auc_meniscus = roc_auc_score(y_score= ypreds_meniscus_soft, y_true= ytrue_meniscus)
# accuracy_meniscus = accuracy_score(y_true= ytrue_meniscus, y_pred= ypreds_meniscus_hard)



logString = "LR: " + str(lr) + "\nMomentum: " + str(momentum) + "\nEpochs: " + str(epochs) + "\nAbnormal MRI AUC: " + str(auc_abnormal) + "\nAbnormal MRI Accuracy: " + str(accuracy_abnormal)+"\n\n" #"\nACL Injury AUC: " + str(auc_acl) + "\nMeniscus Injury AUC: " + str(auc_meniscus) +"\nAbnormal MRI Accuracy: " + str(accuracy_abnormal) + "\nACL Injury Accuracy: " + str(accuracy_acl) + "\nMeniscus Injury Accuracy: " + str(accuracy_meniscus) +"\n\n"

print (logString)

filename = "Logs.txt"
file = open(filename, "a")

file.write(logString)
file.close()



LR: 5e-05
Momentum: 0.6
Epochs: 10
Abnormal MRI AUC: 0.7141052631578948
Abnormal MRI Accuracy: 0.8416666666666667




In [71]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

import torch
print(torch.cuda.is_available())

False
False
True
